<a href="https://colab.research.google.com/github/talgalper/Honours-2021/blob/main/nextstrain_data_treatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install ncbi-blast+
!apt-get install exonerate
!apt-get install dssp
!pip install biostructmap

In [ ]:
import pandas as pd
from collections import defaultdict
import biostructmap

In [ ]:
#read in data file and file in gaps with 0 values
def nextstrain_data_processing(filename, seq_len, gene_id):
    ns_data = pd.read_csv(filename, sep='\t')

    s_gene_data = ns_data[ns_data.gene == gene_id]

    pos = s_gene_data.position

    entropy = s_gene_data.entropy

    pos_to_entropy = defaultdict(float, zip(pos, entropy))

    len_of_ref_seq = seq_len

    list_of_values = [pos_to_entropy[i+1] for i in range(len_of_ref_seq)]
    return list_of_values

In [ ]:
s_protein_data = nextstrain_data_processing('/content/drive/MyDrive/Nextstrain_data/Monthly/19nextstrain_ncov_global_diversity_Jun_2021.tsv', 1273, 'S')

In [ ]:
#check
print(s_protein_data)

In [ ]:
#removes fasta file header
def ref_seq_fasta_format(fasta_filename):
    with open(fasta_filename) as f:
        seq_lines = f.readlines()
    
    formatted_seq_lines = seq_lines[1:]
    
    new_list_lines_removed = list(map(str.strip, formatted_seq_lines)) 
    complete_format = ''.join(new_list_lines_removed)
    return complete_format

In [ ]:
ref_seq = ref_seq_fasta_format('/content/drive/MyDrive/Nextstrain_data/Reference_Sequence/sars_cov_2_spike_ref_seq.fasta')

In [ ]:
#check
ref_seq

In [ ]:
#Biostruct map initalisation
def biostructmap_analysis(pdb_structure, gene_name, input_data, ref_sequence, output_filename):
    structure = biostructmap.Structure(pdb_structure, gene_name)

    data = {'A': input_data, 
            'B': input_data, 
            'C': input_data
            }
 
    ref_seqs = {'A': ref_sequence,
                'B': ref_sequence,
                'C': ref_sequence
                }

    results = structure.map(data, method='default', ref=ref_seqs, radius=15, rsa_range=(0.2, 1.0))

    results.write_data_to_pdb_b_factor(fileobj=output_filename, scale_factor=100)

In [ ]:
biostructmap_analysis('/content/drive/MyDrive/Nextstrain_data/Reference_Sequence/6xr8.1.pdb', 'spike_protein', s_protein_data, ref_seq, '19biostructmap_diversity_jun_2021.pdb')